## Wiener Diarum Sterbelisten
-----------------------------------------------------------------------------------------------

### Data

The digitalised Wiener Zeitung is partially available as TEI XML as well as an extracted HTML on Github. Starting from 1703, there is representative data.This data set contains only a few newspapers per year starting from 1706.

### Task

Filter out Toponyms from the Sterbelisten. For this sequence labeling task we need to first tokenize the words and use the rules (patterns and keywords) made by the team (Nina) to fit to our classes (Toponym or not).


##### What we have 
There is a list with sucessfully extracted toponyms, which are already normalised orthography.
There is a list with rules which are combines Regex and other expressions. 
There is an html file that has \<mark> on most toponyms but sometimes on persons and other names too.
There are defined classes (Toponym/no Toponym).
There are .csv files with Toponyms extracted from Wien Geschichte Wiki.

##### What we want
A trainig and test set containing three matrixes:
T Matrix containing the classes and the rules.(np.matrix, sparse)
Z Matrixes each containing the samples and the rules.(np.matrix, sparse)
X Matrix containing the samples (data frame)


##### What to do

**1. Read in the html file**
Strip of tags and clean the text with re
Expand abbreviations- if possible?
create a list with each sentence as an item

**2. Read in already extracted placenames**
Extract all \<mark> words from HTML**
Using their code
clean the placenames
create a set from exsiting placenames

**3. Building training data**
As there is no clean manual training data, it has to be build:

**X Matrx**
list with all sentenses

maybe spacy?

**T Matrix**

np.sparse matirx 1 colum 0, 1 colum 1, rows are keywords

**Z Matrices**

Each sentense is going to be its own Z Matrix. 
create a list with each word as an item
create a list(/dict if index) with bigrams as items
create a list(/dict if index) with trigrams as items
create a data frame
colums= keywords 
rows = words in sentense



**5. Building test data** 

manually checked! 
There is the 'timemachine_evaluatuin_v1_edited_corrected.jsonl which should contain manually corrected place names but I don't understand where the actual word is tagged or how to filter out that information.

### 1. Read in HTML

In [17]:
'''Import libraries and load html file'''

import os
import re
import codecs

file = codecs.open('annotations_3-21_v4.html', "r", "utf-8")
sterbelisten_html = file.read()

'''Cleaning the text first from both html taggs and xml tags and than in a second step clean the intro and replace with /n'''

sterbelisten_strip_html_1 = re.sub(r"<h2>.*</h2><h3>.*xml \|.*\d+</h3><br/>","",sterbelisten_html)

sterbelisten_strip_html1 = re.sub(r"<hr/>|<p>|</p>|#+|=| =|<html>|</html>|\r|\b \.|\b# \.|(|)","",sterbelisten_strip_html_1)#<mark>|</mark>

sterbelisten_strip_html2 = re.sub(r"    \b","",sterbelisten_strip_html1)

In [18]:
'''create an empty list and split the text at line break, creating von element for each sentense'''

sterbeliste = []
sterbeliste = sterbelisten_strip_html2.split("\n\n\n")
print(sterbeliste)

'''
create a list with words for each sentence
removing empty objects in sterbeliste_words
'''
sterbeliste_words= [i.split(" ") for i in sterbeliste]
sterbeliste_words = [list(filter(None, sublist_words)) for sublist_words in sterbeliste_words]

#print(sterbeliste_words)

['\nDem Peter Frost / einem Cammer im <mark>Greiseckeris  Hauß</mark> im <mark>Diener  Gäßl</mark> / sein Kind Frantz / alt 6 viertl Jahr ', 'Der Maria Nauitschanin / einer Burgerl Wittib im <mark>Primis  Hauß</mark> auf der <mark>Wendelstadt</mark> / ihr Kind Carl / alt 5 Jahr ', 'Christina Kochin / ein Wittib im <mark>Barbieris  Hauß</mark> auf der <mark>Laimgrueben</mark> / alt 67 Jahr ', 'Gottlieb Rabel / ein gewester Haußmeister beyn <mark>3  Mohren</mark> in der <mark>Roßau</mark> / alt 52 Jahr ', 'Dominica Stephanebrin / ein lediges Mensch beym <mark>weissen Ochsen</mark> bey <mark>St  Ulrich</mark> / alt 17 Jahr ', 'Mathias Rausch / ein Fuhrknecht beyn <mark>3  Hertzen</mark> am <mark>Spittelberg</mark> / alt 18 Jahr ', 'Barbara N . ein arme Wittib / <mark>ins   Hauß</mark> in der <mark>Leopold  Stadt</mark> / alt 86 Jahr ', 'Dem Johann Molsheim / ein Handelsman beyn <mark>drey Haasen</mark> bey <mark>St  Ulrich</mark> / sein Kind Johann / alt 5 Jahr ', 'Dem Simon Deix / eine M

In [21]:
from typing import List

def get_location_id(word: str, locations: List) -> int:
    if word not in locations:
        locations[word]= len(locations)
    return locations[word]

num_empty_sentences = 0
empty_sentences =[]
locations={}
samples = []
matched_locations=[]
for sentense in sterbeliste[:1]:
    labels=[]
    # pattern = '<mark>(.*?)</mark>'
    sentense = re.sub(r"\(|\)","",sentense)
    sentense = sentense.split(" ")
    sentense = list(filter(None,sentense))
    if_loc= False
    print(f"Sentence: {sentense}")
    
    for word in sentense:
        if word.find('<mark>') != -1:
            if_loc= True
            location = [word.strip('<mark>')]
        elif word.find('</mark>') != -1:
            location.append(word.strip('<mark>'))
            labels +=[get_location_id(word, locations)] * len(location)
            if_loc= False
        else:
            if if_loc:
                location.append(word.strip('<mark>'))
            else:
                labels.append(None)
    samples.append(labels)
print(f"Length of the sample: {len(sentense)}") 
print(f"Labels: {labels}")
assert len(sentense) == len(labels)

print([(word, label) for (word, label) in zip(sentense, labels)])

Sentence: ['\nDem', 'Peter', 'Frost', '/', 'einem', 'Cammer', 'im', '<mark>Greiseckeris', 'Hauß</mark>', 'im', '<mark>Diener', 'Gäßl</mark>', '/', 'sein', 'Kind', 'Frantz', '/', 'alt', '6', 'viertl', 'Jahr']
Length of the sample: 21
Labels: [None, None, None, None, None, None, None, 0, 0, None, 1, 1, None, None, None, None, None, None, None, None, None]
[('\nDem', None), ('Peter', None), ('Frost', None), ('/', None), ('einem', None), ('Cammer', None), ('im', None), ('<mark>Greiseckeris', 0), ('Hauß</mark>', 0), ('im', None), ('<mark>Diener', 1), ('Gäßl</mark>', 1), ('/', None), ('sein', None), ('Kind', None), ('Frantz', None), ('/', None), ('alt', None), ('6', None), ('viertl', None), ('Jahr', None)]


In [4]:
num_empty_sentences = 0
empty_sentences =[]
locations={}
samples = []
matched_locations=[]
for sentense in sterbeliste:
    matches = []
    pattern = '<mark>(.*?)</mark>'
    try:
        substring = re.findall(pattern, sentense)
    except:
        num_empty_sentences += 1
        empty_sentences.append(sentense)
    for i in substring:
        toponym= re.sub(r"#+|=| =|-|/|","",i)
        toponym_clean = re.sub(r"\d+\s+J|alt|[a-zA-Z]\s+\d+|v\s+J|\s+J|\s+,|ahr|\s+\.","",toponym)
        toponym_clean1 = re.sub(" +"," ",toponym_clean.lstrip().rstrip())
        if toponym_clean1 not in locations:
            locations[toponym_clean1]= len(locations)
        matches.append(locations[toponym_clean1])
    sentense = re.sub(r"<mark>|</mark>|/|\(|\)","",sentense)
    sentense = sentense.split(" ")
    sentense = list(filter(None,sentense))
    samples.append(sentense)
    matched_locations.append(matches)
    
for index,sentense in enumerate(samples):
    Z_s= np.zeros((len(sentense),len(locations)))
    for i in matched_locations[index]:
        pass

print(matched_locations)

NameError: name 'np' is not defined

## 2. Create list with keywords from HTML

In [ ]:
'''rules by using Keywords'''
from io import StringIO
import os
from lxml import etree

""" Get all the possible toponyms from annotations*.html file """

file_path = os.path.join(".", "annotations_3-21_v4.html")

# The annotations document is not well-formed as XML, so we need to use the HTMLParser
parser = etree.HTMLParser()

with open(file_path, encoding='utf-8') as f:
    raw_html = f.read()

# Parse the raw_html
tree = etree.parse(StringIO(raw_html), parser)

root = tree.getroot()
marks = root.xpath("//mark")

length = len(marks)

# Construct a set of toponyms (thus removing duplicates)
toponym_strings = {mark.text for mark in marks}

'''Create a list with all words that were marked with mark'''

toponymstring=[]

'''
clean the strings-> does no work yet
'''
long_toponyms = 0
for mark in marks:
        toponym= re.sub(r"#+|=| =|-|/|\.|ahr","",mark.text)### why does this not filter out f.ex. "Laimgr   50  J"  
        toponym_clean = re.sub(r"\d+\s+J|alt|[a-zA-Z]\s+\d+|v\s+J|\s+J|\s+,|ahr","",toponym)
        toponym_clean1 = re.sub(" +"," ",toponym_clean.lstrip().rstrip())
        toponymstring.append(toponym_clean)
        if toponym_clean1.count(" ") > 2:
            print(toponym_clean1)
            long_toponyms += 1
print(f"Number of long toponyms: {long_toponyms}")
print(len(toponymstring))

# create a set so that the iteration later is easier
#toponym_set= set(toponymstring)
#print(toponym_set)

How to handle Toponyms with more than 2 words - most of them are not really Toponyms but sentenses, where the closing <mark> tag is missing in the html file. 
    
One solution could be exluding the 170 items from the toponym_set, or check them manually. 
    
There are some cleaning issues with spaces after words.

## 3.Building training data

In [ ]:
'''
X Matrix
Dimensions sentesesxsentenses (one column sample df, len(sample))

'''

import pandas as pd

X = pd.DataFrame(sterbeliste)


**T Matrix**

np.sparse matirx 1 colum 0, 1 colum 1, rows are keywords


In [ ]:
'''
T Matrix
Dimensions keywords x 2

'''
import numpy as np
a= np.zeros(len(toponym_set))
b= np.full(len(toponym_set),1)

T = np.stack((b,a.T))
T.T.shape


**Z Matrices**

Each sentense is going to be its own Z Matrix. 
create a list with each word as an item
create a list(/dict if index) with bigrams as items
create a list(/dict if index) with trigrams as items
create a data frame
colums= keywords 
rows = words in sentense

In [ ]:
'''building Z Matrices'''


for sentense in sterbeliste: 
    words = sentense.split(" ")
    words = [list(filter(None,words))]
    Z_S=np.zeros((len(words),len(toponym_set))
    #    for keyword in toponym_set:
    #        kw= [keyword.split("")]
    #        if len(kw) == 1
    #            for w in words:
    #                if w == kw
    #                     match_w= words[key]
    #                     Z_S(kw,w)=1
    #        if len(kw) == 2 
    #             for bigram in bigrams:
    #                 if bigram == kw[0]+' '+kw[1]
    #                     match_w= bigram[keys]
    #                     Z_S((kw[0],kw[1]),w[0],w[1])= 1
 

In [ ]:
'''
building Z Matrices

Loop through each sentense:
first split the sentense into words
create a list with all words
create a list with all bigrams
create a list with all trigrams

Compare keywords to lists and set value to 1 in Z_s

    for each rule j in keywords r:
       if /j/ == 1:
            if j in w? 
                 yes -> 
                     collect the matched words ids w_j
                     z_s(w_j, j) = 1
        if /j/ == 2:
           if j in tuples?
                 yes -> 
                      collect the matched words ids w_j
                      z_s(w_j, j) = 1
'''

for sentense in sterbeliste:
    words= sentense.split(' ')
    words=list(filter(None,words))
    Z_s= np.zeros((len(words),len(toponym_set)))
    bi_gram={}
    for i in range(len(words)-1): 
        bi_gram[words[i],words[i+1]]= f"{i}_{i+1}"
    tri_gram={}
    for i in range(len(words)-2):
        tri_gram[words[i], words[i+1], words[i+2]] =f"{i}_{i+1}_{i+2}"
    for toponym in toponym_set:
        print(toponym)
        toponym = toponym.split(' ')
        #if len(toponym)== 1:
            #print(toponym)
        #if len(toponym)== 2:
            #print(toponym)
 

### create a tokenized sentenses with spacy
import spacy 
from spacy.lang.de.examples import sentences

nlp = spacy.load("de_core_news_sm")

#for i in sterbeliste:
    #doc = nlp(i)
    #print(doc.text)
    #for token in doc:
        #print(token.text, token.pos_, token.dep_)

In [ ]:
'''building Z Matrices'''


for sentense in sterbeliste: 
    words = sentense.split(" ")
    words = [list(filter(None,words))]
    Z_S=np.zeros((len(words),len(toponym_set))
    #    for keyword in toponym_set:
    #        kw= [keyword.split("")]
    #        if len(kw) == 1
    #            for w in words:
    #                if w == kw
    #                     match_w= words[key]
    #                     Z_S(kw,w)=1
    #        if len(kw) == 2 
    #             for bigram in bigrams:
    #                 if bigram == kw[0]+' '+kw[1]
    #                     match_w= bigram[keys]
    #                     Z_S((kw[0],kw[1]),w[0],w[1])= 1
 

In [ ]:
'''
building Z Matrices

Loop through each sentense:
first split the sentense into words
create a list with all words
create a list with all bigrams
create a list with all trigrams

Compare keywords to lists and set value to 1 in Z_s

    for each rule j in keywords r:
       if /j/ == 1:
            if j in w? 
                 yes -> 
                     collect the matched words ids w_j
                     z_s(w_j, j) = 1
        if /j/ == 2:
           if j in tuples?
                 yes -> 
                      collect the matched words ids w_j
                      z_s(w_j, j) = 1
'''

for sentense in sterbeliste:
    words= sentense.split(' ')
    words=list(filter(None,words))
    Z_s= np.zeros((len(words),len(toponym_set)))
    bi_gram={}
    for i in range(len(words)-1): 
        bi_gram[words[i],words[i+1]]= f"{i}_{i+1}"
    tri_gram={}
    for i in range(len(words)-2):
        tri_gram[words[i], words[i+1], words[i+2]] =f"{i}_{i+1}_{i+2}"
    for toponym in toponym_set:
        print(toponym)
        toponym = toponym.split(' ')
        #if len(toponym)== 1:
            #print(toponym)
        #if len(toponym)== 2:
            #print(toponym)
 

### create a tokenized sentenses with spacy
import spacy 
from spacy.lang.de.examples import sentences

nlp = spacy.load("de_core_news_sm")

#for i in sterbeliste:
    #doc = nlp(i)
    #print(doc.text)
    #for token in doc:
        #print(token.text, token.pos_, token.dep_)

In [ ]:
'''building Z Matrices'''


for sentense in sterbeliste: 
    words = sentense.split(" ")
    words = [list(filter(None,words))]
    Z_S=np.zeros((len(words),len(toponym_set))
    #    for keyword in toponym_set:
    #        kw= [keyword.split("")]
    #        if len(kw) == 1
    #            for w in words:
    #                if w == kw
    #                     match_w= words[key]
    #                     Z_S(kw,w)=1
    #        if len(kw) == 2 
    #             for bigram in bigrams:
    #                 if bigram == kw[0]+' '+kw[1]
    #                     match_w= bigram[keys]
    #                     Z_S((kw[0],kw[1]),w[0],w[1])= 1
 

In [ ]:
'''
building Z Matrices

Loop through each sentense:
first split the sentense into words
create a list with all words
create a list with all bigrams
create a list with all trigrams

Compare keywords to lists and set value to 1 in Z_s

    for each rule j in keywords r:
       if /j/ == 1:
            if j in w? 
                 yes -> 
                     collect the matched words ids w_j
                     z_s(w_j, j) = 1
        if /j/ == 2:
           if j in tuples?
                 yes -> 
                      collect the matched words ids w_j
                      z_s(w_j, j) = 1
'''

for sentense in sterbeliste:
    words= sentense.split(' ')
    words=list(filter(None,words))
    Z_s= np.zeros((len(words),len(toponym_set)))
    bi_gram={}
    for i in range(len(words)-1): 
        bi_gram[words[i],words[i+1]]= f"{i}_{i+1}"
    tri_gram={}
    for i in range(len(words)-2):
        tri_gram[words[i], words[i+1], words[i+2]] =f"{i}_{i+1}_{i+2}"
    for toponym in toponym_set:
        print(toponym)
        toponym = toponym.split(' ')
        #if len(toponym)== 1:
            #print(toponym)
        #if len(toponym)== 2:
            #print(toponym)
 

In [ ]:
### create a tokenized sentenses with spacy
import spacy 
from spacy.lang.de.examples import sentences

nlp = spacy.load("de_core_news_sm")

#for i in sterbeliste:
    #doc = nlp(i)
    #print(doc.text)
    #for token in doc:
        #print(token.text, token.pos_, token.dep_)

## 5. Building test data

In [ ]:
'''Loading manually checked test data'''

with jsl.open('timemachine_evaluation_v1_edited_corrected.jsonl','r') as reader: #read in jsonl with jsonlines
    df = pd.json_normalize(reader) 

### I don't understand where the tagged toponym is or whether there is any